In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# Load tidied dataframes

# spells_df = pd.read_pickle('data/spells_df')
monster_df = pd.read_pickle('data/monster_df')
actions_df = pd.read_pickle('data/actions_df')
spellbook_df = pd.read_pickle('data/spellbook_df')
innate_spellbook_df = pd.read_pickle('data/innate_spellbook_df')
# special_abilities_df = pd.read_pickle('data/special_abilities_df')
# condition_immunities_df = pd.read_pickle('data/condition_immunities_df')
# damage_immunities_df = pd.read_pickle('data/damage_immunities_df')
# damage_resistances_df = pd.read_pickle('data/damage_resistances_df')
# damage_vulnerabilities_df = pd.read_pickle('data/damage_vulnerabilities_df')

# `actions_df`

The following creatures have `multiattack` actions which are formatted differently then the others

In [26]:
actions_df.at[('Animated Armor', 'Multiattack'), 'desc'] = \
    'The armor makes two slam attacks.'
actions_df.at[('Cult Fanatic', 'Multiattack'), 'desc'] = \
    'The fanatic makes two melee dagger attacks.'
actions_df.at[('Deva', 'Multiattack'), 'desc'] = \
    'The deva makes two mace attacks.'
actions_df.at[('Doppleganger', 'Multiattack'), 'desc'] = \
    'The doppelganger makes two slam attacks.'
actions_df.at[('Erinyes', 'Multiattack'), 'desc'] = \
    'The erinyes makes three attacks, either with its longsword or its longbow.'
actions_df.at[('Gladiator', 'Multiattack'), 'desc'] = \
    'The gladiator makes three spear melee attacks or two ranged spear attacks.'
actions_df.at[('Iron Golem', 'Multiattack'), 'desc'] = \
    'The golem makes two melee attacks: either with its sword or its slam attack.'
actions_df.at[('Knight', 'Multiattack'), 'desc'] = \
    'The knight makes two greatsword attacks.'
actions_df.at[('Planetar', 'Multiattack'), 'desc'] = \
    'The planetar makes two greatsword attacks.'
actions_df.at[('Scout', 'Multiattack'), 'desc'] = \
    'The scout makes two shortsword attacks or two longbow attacks.'
actions_df.at[('Spy', 'Multiattack'), 'desc'] = \
    'The spy makes two shortsword attacks.'
actions_df.at[('Thug', 'Multiattack'), 'desc'] = \
    'The thug makes two mace attacks.'

In [4]:
df = actions_df.reset_index('action')

## extracting multiattacks

In [5]:
def normalize_multiattack(x):
    df = x.set_index('action')
    if 'Multiattack' not in df.index:
        return
    desc = df.loc['Multiattack'].desc.lower().strip()
    actions = [format_action(action) for action in df.index
               if action != 'Multiattack']
    multiattack, exception = extract_attacks(desc, actions)
    return pd.DataFrame({'desc': [desc],
                         'multiattack': [[multiattack]],
                         'exception': [exception]})


def format_action(action):
    action = action.lower()
    i = action.find('(')
    if i > 0:
        return action[:i-1]
    else:
        return action


def extract_attacks(desc, actions):
    exceptions_pattern = r'\b({})\b'.format('|'.join(x for x in exceptions))
    exception = re.search(exceptions_pattern, desc) is not None
    actions_pattern = '|'.join(x for x in actions)
    split_pattern = r'.*?({})'.format(actions_pattern)
    numerics_pattern = '|'.join(x for x in numerics.keys())
    multiattack = list()
    for x in re.finditer(split_pattern, desc):
        start, end = x.span()
        x = re.findall(numerics_pattern, desc[start:end])
        if x:
            n = numerics[x[-1]]
        else:
            n = 1
        y = re.findall(actions_pattern, desc[start:end])
        multiattack.extend(n * y)
    return multiattack, exception


numerics = {'one': 1,
            'two': 2,
            'twice': 2,
            'three': 3,
            'four': 4,
            'six': 6,
            'seven': 7}
exceptions = ('or', 'alternatively', 'when', 'in place', 'replace',
              'also', 'as many', 'only', r'\dd\d', 'with a different weapon')

In [6]:
multiattacks_df = df.groupby('name').apply(normalize_multiattack)
multiattacks_df.reset_index(level=1, drop=True, inplace=True)

We'll correct the following exceptions by hand:

In [11]:
[x[0] for x in multiattacks_df[multiattacks_df.exception].iterrows()]

['Bandit Captain',
 'Barbed Devil',
 'Centaur',
 'Chimera',
 'Chuul',
 'Dragon Turtle',
 'Drider',
 'Efreeti',
 'Erinyes',
 'Glabrezu',
 'Gladiator',
 'Half-Red Dragon Veteran',
 'Horned Devil',
 'Hydra',
 'Iron Golem',
 'Kraken',
 'Lamia',
 'Lizardfolk',
 'Manticore',
 'Medusa',
 'Merrow',
 'Oni',
 'Sahuagin',
 'Scout',
 'Shambling Mound',
 'Vampire Spawn',
 'Veteran',
 'Violet Fungus',
 'Werebear',
 'Wight',
 'Wyvern']

In [25]:
multiattacks_df.at['Bandit Captain', 'multiattack'] = [['scimitar', 'scimitar', 'dagger'],
                                                       ['dagger', 'dagger']]
multiattacks_df.at['Barbed Devil', 'multiattack'] = [['tail', 'claw', 'claw'],
                                                      ['hurl flame', 'hurl flame']]
multiattacks_df.at['Centaur', 'multiattack'] = [['pike', 'hooves'],
                                                 ['longbow', 'longbow']]
multiattacks_df.at['Chimera', 'multiattack'] = [['bite', 'horns', 'claws'],
                                                 ['fire breath', 'horns', 'claws'],
                                                 ['bite', 'fire breath', 'claws']]
multiattacks_df.at['Chuul', 'multiattack'] = [['pincer', 'pincer', 'tentacles'],
                                               ['pincer', 'pincer']]
multiattacks_df.at['Dragon Turtle', 'multiattack'] = [['bite', 'claw', 'claw'],
                                                       ['bite', 'tail']]
multiattacks_df.at['Drider', 'multiattack'] = [['longsword', 'longsword', 'longsword'],
                                                ['longsword', 'longsword', 'bite'],
                                                ['longbow', 'longbow', 'longbow'],
                                                ['longbow', 'longbow', 'bite']]
multiattacks_df.at['Efreeti', 'multiattack'] = [['scimitar', 'scimitar'],
                                                 ['hurl flame', 'hurl flame']]
multiattacks_df.at['Erinyes', 'multiattack'] = [['longsword', 'longsword', 'longsword'],
                                                ['longbow', 'longbow', 'longbow']]
multiattacks_df.at['Glabrezu', 'multiattack'] = [['pincer', 'pincer', 'fist', 'fist'],
                                                 ['pincer', 'pincer']] # spells?
multiattacks_df.at['Gladiator', 'multiattack'] = [['spear', 'spear', 'spear'],
                                                  ['spear', 'spear']]
multiattacks_df.at['Half-Red Dragon Veteran', 'multiattack'] = [['longsword', 'longsword'],
                                                                ['longsword', 'longsword', 'shortsword']]
multiattacks_df.at['Horned Devil', 'multiattack'] = [['fork', 'fork', 'tail'],
                                                     ['fork', 'hurl flame', 'tail'],
                                                     ['fork', 'fork', 'hurl flame']]
multiattacks_df.at['Hydra', 'multiattack'] = [['bite', 'bite', 'bite', 'bite', 'bite']]
multiattacks_df.at['Iron Golem', 'multiattack'] = [['sword', 'sword'],
                                                   ['slam', 'slam']]
multiattacks_df.at['Kraken', 'multiattack'] = [['tentacle', 'tentacle', 'tentacle'],
                                               ['tentacle', 'tentacle', 'fling'],
                                               ['tentacle', 'fling', 'fling'],
                                               ['fling', 'fling', 'fling']]
multiattacks_df.at['Lamia', 'multiattack'] = [['claws', 'dagger'],
                                              ['claws', 'intoxicating touch']]
multiattacks_df.at['Lizardfolk', 'multiattack'] = [['bite', 'heavy club'],
                                                   ['bite', 'javelin'],
                                                   ['bite', 'spiked shield'],
                                                   ['heavy club', 'javelin'],
                                                   ['heavy club', 'spiked shield']]
multiattacks_df.at['Manticore', 'multiattack'] = [['bite', 'claw', 'claw'],
                                                  ['tail spike', 'tail spike', 'tail spike']]
multiattacks_df.at['Medusa', 'multiattack'] = [['snake hair', 'shortsword', 'shortsword'],
                                               ['longbow', 'longbow']]
multiattacks_df.at['Merrow', 'multiattack'] = [['bite', 'claws'],
                                               ['bite', 'harpoon']]
multiattacks_df.at['Oni', 'multiattack'] = [['claw', 'claw'],
                                            ['glaive', 'glaive']]
multiattacks_df.at['Sahuagin', 'multiattack'] = [['bite', 'claws'],
                                                 ['bite', 'spear']]
multiattacks_df.at['Scout', 'multiattack'] = [['shortsword', 'shortsword'],
                                              ['longbow', 'longbow']]
multiattacks_df.at['Shambling Mound', 'multiattack'] = [['slam', 'slam'],
                                                        ['slam', 'slam', 'engulf']]
multiattacks_df.at['Vampire Spawn', 'multiattack'] = [['claws', 'claws'],
                                                      ['claws', 'bite']]
multiattacks_df.at['Veteran', 'multiattack'] = [['longsword', 'longsword', 'shortsword'],
                                                ['longsword', 'longsword']]
multiattacks_df.at['Werebear', 'multiattack'] = [['claw', 'claw'],
                                                 ['greataxe', 'greataxe']]
multiattacks_df.at['Wight', 'multiattack'] = [['longsword', 'longsword'],
                                              ['longbow', 'longbow'],
                                              ['life drain', 'longsword']]
multiattacks_df.at['Wyvern', 'multiattack'] = [['bite', 'stinger'],
                                               ['bite', 'claws'],
                                               ['claws', 'stinger']]

# other actions

In [ ]:
def damage_summary(damage_dice, damage_bonus):
    pattern = re.compile(r'(\d+)d(\d+)')
    damage = [np.array([damage_bonus, 0])]
    for dice in re.finditer(pattern, damage_dice):
        x, y = dice.groups()
        damage.append(damage_statistics(int(x), int(y)))
    mean, variance = sum(damage)
    return mean, variance


def damage_statistics(number, size):
    mu = (size + 1) / 2
    mean = number * mu
    var = number * (sum_of_squares(size) / size - mu ** 2)
    return np.array([mean, var])


def sum_of_squares(n):
    return n*(2*n+1)*(n+1)/6

In [ ]:
actions_df.loc['Adult Blue Dracolich', 'Bite'].desc

Some of the breath weapons have to be split up.

In [ ]:
desc = 'Melee Weapon Attack: +12 to hit, reach 15 ft., one target. Hit: 16 (2d8 + 7) bludgeoning damage.'
pattern = re.compile(r'^(.* Attack: .* targets?). (Hit: .* damage.)')
g = re.match(pattern, desc)

In [ ]:
g.span()

In [ ]:
def parse_desc(desc):
    pattern = re.compile(r'^(.* Attack: .* (creature|target)s?)\. (Hit: [^\.]*)')
    g = re.match(pattern, desc)
    if g:
        start, end = g.span()
        attack, _, result = g.groups()
        print (parse_attack(attack), parse_result(result))
    else:
        pass
    return

def parse_attack(attack):
    pattern = re.compile(r'Attack: ' \
                         + r'+(\d+) to hit, ' \
                         + r'(.*)ft\.\W?, ' \
                         + r''
                        )
    return attack


def parse_result(result):
    return ''


actions_df.desc.apply(parse_desc)

In [ ]:
for x in actions_df.iterrows():
    if x[0][1] == 'Multiattack':
        continue
    if not x[1].damage_dice:
        continue
    print(x[0])
    mean, var = damage_summary(x[1].damage_dice,
                               x[1].damage_bonus)
    print(mean, var)
    print('\n')
    # print('mean: {:0.2f}\nstd : {:0.2f}'.format(mean, np.sqrt(var)))

In [ ]:
oddities = ('Adult Brass Dragon - Bite',
            'Ancient Black Dragon - Tail')

In [ ]:
damage_summary('1d6 + 2d4', 4)